# Wine Reviews Recommendation Systems

**Prepared by Elizabeth Webster**

*November 2022*

## Overview

## Business Problem

## Dataset

# Data Understanding

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Data/winemag-data-130k-v2.csv.zip', encoding='latin-1', index_col=0)

In [3]:
df.tail()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr SpÃ¤tlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben MÃ¼ller-Burggraef) 2013...,Riesling,Dr. H. Thanisch (Erben MÃ¼ller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwineÂ,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,GewÃ¼rztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss
129970,France,"Big, rich and off-dry, this is powered by inte...",Lieu-dit Harth CuvÃ©e Caroline,90,21.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Schoffit 2012 Lieu-dit Harth CuvÃ©e Ca...,GewÃ¼rztraminer,Domaine Schoffit


In [4]:
df['taster_name'].value_counts()

Roger Voss            25514
Michael Schachner     15134
Kerin OâKeefe       10776
Virginie Boone         9537
Paul Gregutt           9532
Matt Kettmann          6332
Joe Czerwinski         5147
Sean P. Sullivan       4966
Anna Lee C. Iijima     4415
Jim Gordon             4177
Anne KrebiehlÂ MW      3685
Lauren Buzzeo          1835
Susan Kostrzewa        1085
Mike DeSimone           514
Jeff Jenssen            491
Alexander Peartree      415
Carrie Dykes            139
Fiona Adams              27
Christina Pickard         6
Name: taster_name, dtype: int64

In [22]:
df['number'] = 1

In [23]:
variety_df = df.groupby('variety').sum()

In [24]:
variety_df.head()

,points,price,number
variety,,,
Abouriou,263,105.0,3
Agiorgitiko,5483,1485.0,63
Aglianico,32050,11433.0,359
Aidani,82,27.0,1
Airen,245,27.0,3


In [32]:
variety_df.reset_index(inplace=True)
variety_df.head()

,variety,points,price,number
0,Abouriou,263,105.0,3
1,Agiorgitiko,5483,1485.0,63
2,Aglianico,32050,11433.0,359
3,Aidani,82,27.0,1
4,Airen,245,27.0,3


In [48]:
top_varieties = []
for i in range(len(variety_df)):
    if variety_df['number'][i] > 300:
        top_varieties.append(variety_df['variety'][i])
    else:
        pass

In [53]:
top_varieties

['Aglianico',
 'AlbariÃ±o',
 'Barbera',
 'Bordeaux-style Red Blend',
 'Bordeaux-style White Blend',
 'Cabernet Franc',
 'Cabernet Sauvignon',
 'CarmenÃ¨re',
 'Champagne Blend',
 'Chardonnay',
 'Chenin Blanc',
 'Corvina, Rondinella, Molinara',
 'Gamay',
 'Garnacha',
 'GewÃ¼rztraminer',
 'Glera',
 'Grenache',
 'GrÃ¼ner Veltliner',
 'Malbec',
 'Merlot',
 'Moscato',
 'Nebbiolo',
 "Nero d'Avola",
 'Petite Sirah',
 'Pinot Blanc',
 'Pinot Grigio',
 'Pinot Gris',
 'Pinot Noir',
 'Port',
 'Portuguese Red',
 'Portuguese White',
 'Red Blend',
 'RhÃ´ne-style Red Blend',
 'RhÃ´ne-style White Blend',
 'Riesling',
 'RosÃ©',
 'Sangiovese',
 'Sangiovese Grosso',
 'Sauvignon',
 'Sauvignon Blanc',
 'Shiraz',
 'Sparkling Blend',
 'Syrah',
 'Tempranillo',
 'Tempranillo Blend',
 'Viognier',
 'White Blend',
 'Zinfandel']

In [60]:
df[df['variety'].isin(top_varieties)]

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,number
0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin OâKeefe,@kerinokeefe,Nicosia 2013 VulkÃ Bianco (Etna),White Blend,Nicosia,1
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,1
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwineÂ,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,1
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,1
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwineÂ,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr SpÃ¤tlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben MÃ¼ller-Burggraef) 2013...,Riesling,Dr. H. Thanisch (Erben MÃ¼ller-Burggraef),1
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwineÂ,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation,1
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,GewÃ¼rztraminer,Domaine Gresser,1
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss,1
